SQL에서 응답자 기록 가져오기

In [1]:
import pymysql,pandas as pd

# MySQL Connection 연결
conn = pymysql.connect(host='52.78.100.225',user='pida',password='PidaDaily1!',
                       db='pida_database',charset='utf8')

curs = conn.cursor()

# SQL쿼리문
sql = "select * from pida_database.surveys"
curs.execute(sql)

# 데이터 Fetch
rows = curs.fetchall()

colname = curs.description
# tuple 형태로 데이터 가져온다
col = []

# 컬럼명 추출해서 컬럼 리스트 생성
for i in colname:
    col.append(i[0])
    
# 데이터 프레임 생성하기
emp = pd.DataFrame(list(rows),columns=col)
emp = emp[["id","answer"]].set_index("id")

In [2]:
# 유저의 id와 answer 답변 목록이 적힌 DataFrame
emp.head(3)

,answer
id,
1,"1, 2, 2, 2, 3, 2, 3, 2, 2.5, 4, 2.5, 2.5, 2.5,..."
2,"1, 2, 2, 2, 3, 2, 3, 2, 2.5, 4, 2.5, 2.5, 2.5,..."
3,"1, 2, 2, 2, 3, 2, 3, 2, 2.5, 4, 2.5, 2.5, 2.5,..."


유저의 피부 타입 및 점수 산출

In [3]:
emp["건성지성"] = emp["answer"].apply(lambda x : sum(list(map(float,x.split(", ")[0:7]))))
emp["민감성저항성"] = emp["answer"].apply(lambda x : sum(list(map(float,x.split(", ")[7:16]))))
emp["색소성비색소성"] = emp["answer"].apply(lambda x : sum(list(map(float,x.split(", ")[16:]))))

def 건성지성(x):
    if x <= 13:
        return "건성"
    elif 13 < x <= 17:
        return "중성"
    else:
        return "지성"
    
def 색소성비색소성(x):
    if x < 15:
        return "비색소침착성"
    else:
        return "색소침착성"

def 민감성저항성(x):
    if x <= 15:
        return "저항성"
    else:
        return "민감성"
    
emp["건성지성"] = emp["건성지성"].apply(lambda x : 건성지성(x) + " " +str(x)+"점")
emp["민감성저항성"] = emp["민감성저항성"].apply(lambda x : 민감성저항성(x) + " " +str(x)+"점")
emp["색소성비색소성"] = emp["색소성비색소성"].apply(lambda x : 색소성비색소성(x) + " " +str(x)+"점")

In [11]:
emp.to_csv("설문결과_타입.csv",encoding="utf-8-sig")

로컬에서 질문/선택지 가져오기

In [12]:
survey = pd.read_csv("설문테이블.csv",encoding="utf-8")

In [13]:
# 질문/선택지 정보가 담긴 DataFrame
survey.head(3)

,질문타입,질문번호,질문,1번선택지,2번선택지,3번선택지,4번선택지,5번선택지
0,지성건성,1,"세안 이후 기초제품, 보습제 등 아무것도 바르지 않은 상태에서 2~3시간 후에 양 ...",매우 거칠거나 각질이 일어난다,당기는 느낌이 든다,적당히 촉촉하다,윤기가 흐르고 번들거린다,NaN
1,지성건성,2,"파우더를 사용하지 않고 기초 화장(베이스, 파운데이션)만 한 상태에서 2∼3시간 후...",각질이 일어나거나 주름 사이에 끼어 있다,매끈하다,적당히 광이 난다,윤기가 흐르고 번들거린다,나는 기초화장(파운데이션)을 하지 않는다
2,지성건성,3,거울로 얼굴을 보면 바늘 끝 크기 혹은 더 큰 크기의 커다란 모공이 얼마나 많이 있나요?,없다,T존(이마와 코)에만 약간 있다,많다,대단히 많다,모른다


In [14]:
survey_result = survey[["질문타입","질문번호","질문"]]

import warnings
warnings.filterwarnings("ignore")

# 유저 수만큼 설문에 대한 답이 survey_result 컬럼으로 추가된다
for i in range(emp.shape[0]):
    col = "id" + str(i+1)
    survey_result[col] = ""
    
# answer은 유저가 답변한 답변리스트
# [1,2,3,4,2.5,1,2,3...]
    answer = emp.iloc[i,0].split(", ")
    for j in range(len(answer)):
        if answer[j] != "2.5":
            # n번 선택지에 해당하는 선택지 할당해주기
            survey_result.loc[j,col] = survey.loc[j,answer[j]+"번선택지"]
            
        elif answer[j] == "2.5":
            # 1번 선택지에 해당하는 선택지 할당해주기
            survey_result.loc[j,col] = survey.loc[j,"5번선택지"]
            
survey_result.set_index("질문번호").T

질문번호,1,2,3,4,5,6,7,8,9,10,...,14,15,16,17,18,19,20,21,22,23
질문타입,지성건성,지성건성,지성건성,지성건성,지성건성,지성건성,지성건성,민감성저항성,민감성저항성,민감성저항성,...,민감성저항성,민감성저항성,민감성저항성,색소성비색소성,색소성비색소성,색소성비색소성,색소성비색소성,색소성비색소성,색소성비색소성,색소성비색소성
질문,"세안 이후 기초제품, 보습제 등 아무것도 바르지 않은 상태에서 2~3시간 후에 양 ...","파우더를 사용하지 않고 기초 화장(베이스, 파운데이션)만 한 상태에서 2∼3시간 후...",거울로 얼굴을 보면 바늘 끝 크기 혹은 더 큰 크기의 커다란 모공이 얼마나 많이 있나요?,자신이 느끼는 얼굴 피부는 다음 중에서 어떤 상태인가요?,클렌징폼으로 세안한 직후 얼굴 피부는 어떤가요?,얼굴의 T존(이마와 코)에 기름기가 있나요?,보습제를 바르고 2∼3시간 이후에 양 볼은 어떤가요?,얼굴에 붉게 돌출된 트러블이 발생한 경험이 있나요?,"피부 관리용 제품(클렌저, 보습제, 색조화장품, 및 화장품 등을 포함)을 사용하면 ...",과거에 여드름에 대한 진단을 받은 적이 있나요?,...,얼굴이나 코에 붉거나 푸른색의 혈관이 얼마나 많이 보이나요?,"화장품, 자외선 차단제, 피부 관리용 제품을 사용한 후에 피부의 발적, 가려움증, ...","가족 중에서 아토피 피부염, 습진, 천식, 알레르기로 진단받은 분이 있나요?",트러블 발생 후 혹은 피부 속으로 파고 들어간 모발에 의해 짙은 갈색이나 검은 점이...,"자주 접히는 피부(팔꿈치, 겨드랑이 등)에 색소침착이 있나요?",과거에 얼굴 기미(옅거나 짙은 갈색 혹은 회색 반점)로 진단받은 적이 있거나 현재 ...,"얼굴, 가슴, 등, 혹은 양 팔에 작은 갈색 점(주근깨, 일광 흑자)이 있거나, 있...",며칠 동안 계속해서 햇빛에 노출되면 어떤 일이 일어났나요?,"햇볕에 나가면, 주근깨(작은 1∼2 mm 크기의 편평한 점)가 생기나요?",부모님 중의 어느 한 분이라도 주근깨가 있나요?
id1,매우 거칠거나 각질이 일어난다,매끈하다,T존(이마와 코)에만 약간 있다,중성,정상적인 느낌이다,가끔 있다,약간 윤기가 흐른다,거의 없다,나는 얼굴에 피부 관리용 제품을 바르지 않는다.,심각한 경우에 해당한다,...,많이 보인다(코를 포함한 얼굴 전체에서 7개 이상).,이런 제품을 사용하지 않는다,잘 모르겠다,전혀 생기지 않는다,전혀없다.,없다.,없다.,햇빛에 타서 물집이 생겼으나 피부색은 변하지 않는다.,전혀 생기지 않는다,없다.
id2,매우 거칠거나 각질이 일어난다,매끈하다,T존(이마와 코)에만 약간 있다,중성,정상적인 느낌이다,가끔 있다,약간 윤기가 흐른다,거의 없다,나는 얼굴에 피부 관리용 제품을 바르지 않는다.,심각한 경우에 해당한다,...,많이 보인다(코를 포함한 얼굴 전체에서 7개 이상).,이런 제품을 사용하지 않는다,잘 모르겠다,전혀 생기지 않는다,전혀없다.,없다.,없다.,햇빛에 타서 물집이 생겼으나 피부색은 변하지 않는다.,전혀 생기지 않는다,없다.
id3,매우 거칠거나 각질이 일어난다,매끈하다,T존(이마와 코)에만 약간 있다,중성,정상적인 느낌이다,가끔 있다,약간 윤기가 흐른다,거의 없다,나는 얼굴에 피부 관리용 제품을 바르지 않는다.,심각한 경우에 해당한다,...,많이 보인다(코를 포함한 얼굴 전체에서 7개 이상).,이런 제품을 사용하지 않는다,잘 모르겠다,전혀 생기지 않는다,전혀없다.,없다.,없다.,햇빛에 타서 물집이 생겼으나 피부색은 변하지 않는다.,전혀 생기지 않는다,없다.
id4,매우 거칠거나 각질이 일어난다,매끈하다,많다,건성,약간 건조한 느낌이 들지만 갈라지는 느낌은 없다,전혀 없다,"매우 거칠거나, 각질이 일어난다",전혀 없다,전혀없다,친구와 지인이 나에게 이런 증상이 있다고 얘기한다,...,없다,자주 있다,가족 중 한 명이 진단 받았다,전혀 생기지 않는다,전혀없다.,"한 번 있었지만, 소실되었다",없다.,햇빛에 타서 물집이 생겼으나 피부색은 변하지 않는다.,매년 약간씩 작은 주근깨가 생긴다,얼굴에 많이 있었다
id5,적당히 촉촉하다,윤기가 흐르고 번들거린다,대단히 많다,복합,기름기가 있는 느낌이다,가끔 있다,윤기가 흐르고 번들거린다. 혹은 보습제의 필요성을 못 느껴서 사용하지 않는다,최소한 1개월에 1회 있다,항상 있다,진단을 받았다,...,약간 보인다(코를 포함한 얼굴 전체에서 4∼6개).,항상 있다,"가족 중 많은 사람이 피부염, 습진, 천식, 알레르기를 가지고 있다",여드름이나 피부 속으로 파고 들어간 모발이 전혀 없다.,짙게 있다,"한 번 있었지만, 소실되었다",약간 있다(1∼5개),피부색이 약간 짙어졌다.,자주 주근깨가 생긴다,"얼굴, 가슴, 목, 양쪽 어깨에 많이 있었다"
id6,당기는 느낌이 든다,적당히 광이 난다,T존(이마와 코)에만 약간 있다,건성,약간 건조한 느낌이 들지만 갈라지는 느낌은 없다,가끔 있다,약간 윤기가 흐른다,최소한 1개월에 1회 있다,거의 없다,없다,...,드물게 보인다(코를 포함한 얼굴 전체에서 1∼3개).,때때로 있다,가족 중 한 명이 진단 받았다,전혀 생기지 않는다,약간 있는 편이다,없다.,약간 있다(1∼5개),피부색이 약간 짙어졌다.,매년 약간씩 작은 주근깨가 생긴다,얼굴에 약간 있었다.
id7,당기는 느낌이 든다,적당히 광이 난다,T존(이마와 코)에만 약간 있다,건성,약간 건조한 느낌이 들지만 갈라지는 느낌은 없다,가끔 있다,약간 윤기가 흐른다,최소한 1개월에 1회 있다,거의 없다,없다,...,드물게 보인다(코를 포함한 얼굴 전체에서 1∼3개).,때때로 있다,가족 중 한 명이 진단 받았다,전혀 생기지 않는다,약간 있는 편이다,없다.,약간 있다(1∼5개),피부색이 약간 짙어졌다.,매년 약간씩 작은 주근깨가 생긴다,얼굴에 약간 있었다.
id8,당기는 느낌이 든다,적당히 광이 난다,T존(이마와 코)에만 약간 있다,건성,약간 건조한 느낌이 들지만 갈라지는 느낌은 없다,가끔 있다,약간 윤기가 흐른다,최소한 1개월에 1회 있다,거의 없다,없다,...,드물게 보인다(코를 포함한 얼굴 전체에서 1∼3개).,때때로 있다,가족 중 한 명이 진단 받았다,전혀 생기지 않는다,약간 있는 편이다,없다.,약간 있다(1∼5개),피부색이 약간 짙어졌다.,매년 약간씩 작은 주근깨가 생긴다,얼굴에 약간 있었다.


In [15]:
survey_result.to_csv("설문결과_문항.csv",encoding="utf-8-sig")